In [68]:
import joblib
import ast
from openai import BaseModel
import pandas as pd

class ParseResult(BaseModel):
    tags: list[str]

df = joblib.load('/Users/bill/experiments/enron-analysis/final-1727115257.joblib')
df['From'] = df['From'].apply(lambda x: ast.literal_eval(x) if x == x else [])
df['To'] = df['To'].apply(lambda x: ast.literal_eval(x) if x == x else [])
df['Date'] = pd.to_datetime(df['Date'])
df = df[
	df['result'].apply(lambda x: x['result']).notnull()
]
df['num_tags'] = df['result'].apply(lambda x: len(x['result'].tags))
all_tags = set(tag for tags in df['result'].apply(lambda r: r['result'].tags) for tag in tags)
for tag in all_tags:
	df[tag] = df['result'].apply(lambda x: tag in x['result'].tags)

In [61]:
all_tags

{' LOGISTICS',
 '911',
 '>',
 'BUSINESS',
 'FINANCIAL',
 'FRAUD',
 'FUNNY',
 'GOSSIP',
 'GUILT',
 'HARASSMENT',
 'LEGAL_ADVICE',
 'LEGAl_ADVICE',
 'LOGISTICS',
 'MENTORSHIP',
 'PERSONAL',
 'POLITICS',
 'PROMOTIONAL',
 'ROMANCE',
 'SPAM'}

In [71]:
tag_list = list(all_tags)
for x in df[df['FRAUD']].groupby(tag_list):
	tags_in_group = [t for idx, t in enumerate(tag_list) if x[0][idx]]
	print(tags_in_group)

['FRAUD']
['PERSONAL', 'FRAUD']
['FUNNY', 'FRAUD']
['FUNNY', 'PERSONAL', 'FRAUD']
['LEGAL_ADVICE', 'FRAUD']
['LEGAL_ADVICE', 'BUSINESS', 'FRAUD']
['911', 'FRAUD']
['911', 'LEGAL_ADVICE', 'FRAUD']
['ROMANCE', 'FRAUD']
['LOGISTICS', 'FRAUD']
['LOGISTICS', 'PERSONAL', 'FRAUD']
['LOGISTICS', 'FUNNY', 'PERSONAL', 'FRAUD']
['LOGISTICS', 'LEGAL_ADVICE', 'FRAUD']
['LOGISTICS', 'ROMANCE', 'PERSONAL', 'FRAUD']
['GUILT', 'FRAUD']
['GUILT', 'FUNNY', 'FRAUD']
['GUILT', 'LEGAL_ADVICE', 'FRAUD']
['GUILT', 'ROMANCE', 'PERSONAL', 'FRAUD']
['GUILT', 'LOGISTICS', 'LEGAL_ADVICE', 'FRAUD']


In [112]:
import os

def handlena(val, default):
	return val if val == val else default

def dump_to_disk(df, dir, name_fn = None):
# 	From: Herndon, Wayne [mailto:wherndon@tractebelusa.com]
# Sent: Thursday, February 08, 2001 1:32 PM
# To: 'missy@vollmerpr.com'
# Subject:

	if name_fn is None:
		name_fn = lambda x: x['Message-ID']

	os.makedirs(dir, exist_ok=True)
	for _, row in df.iterrows():
		date_pretty = row['Date'].strftime('%A, %B %d, %Y %I:%M %p')
		metadata = f"""Message-ID: {row['Message-ID']}
From: {'; '.join(handlena(row['From'], []))}
Sent: {date_pretty}
To: {'; '.join(handlena(row['To'], []))}
Subject: {handlena(row['Subject'], '')}"""
		with open(os.path.join(dir, f"{name_fn(row)}.txt"), 'w') as f:
			f.write(metadata)
			f.write('\n\n')
			f.write(row['content'])

# os.makedirs('/Users/bill/experiments/enron-analysis/outputs', exist_ok=True)

# tag_name = 'FRAUD'

# for tag_name in ['FRAUD', 'ROMANCE']:
# 	output_dir = f'/Users/bill/experiments/enron-analysis/outputs/{tag_name}'

# 	filter = lambda df: ~df['content'].str.contains('To Mr. Ken Lay,', case=False)

# 	tag_list = list(all_tags)
# 	# for x in df[df[tag_name]].groupby(tag_list):
# 	# 	tags_in_group = [t for idx, t in enumerate(tag_list) if x[0][idx]]
# 		# print(tags_in_group)

# 	for x in df[df[tag_name]].groupby(tag_list):
# 		tags_in_group = [t for idx, t in enumerate(tag_list) if x[0][idx]]
# 		tags_in_group.sort()
# 		dir_name = '-'.join(tags_in_group)
# 		condition = df[tags_in_group[0]]
# 		for other_condition in tags_in_group[1:]:
# 			# condition &= df[other_condition]
# 			condition = (condition & df[other_condition])
# 		if filter:
# 			condition = condition & filter(df)

# 		dump_to_disk(df[condition], os.path.join(output_dir, dir_name))

In [67]:
df[df['FRAUD']]

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,result,content_no_attachment,user_prompt,token_count_total,num_tags,GUILT,LEGAl_ADVICE,LOGISTICS,ROMANCE,911,LEGAL_ADVICE,GOSSIP,MENTORSHIP,FUNNY,SPAM,PROMOTIONAL,PERSONAL,BUSINESS,FINANCIAL,>,POLITICS,LOGISTICS,HARASSMENT,FRAUD


In [16]:
df['num_tags'] = df['result'].apply(lambda x: len(x['result'].tags))

In [ ]:
for c in df.sort_values('num_tags', ascending=False).head()['content']:
	print(c)
	print('-------------------')
	print('-------------------')
	print('-------------------')
	print('-------------------')
	print('-------------------')
	print('-------------------')
	print()

In [30]:
df.head()

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,...,SPAM,PROMOTIONAL,PERSONAL,BUSINESS,FINANCIAL,>,POLITICS,LOGISTICS,HARASSMENT,FRAUD
0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00+00:00,['phillip.allen@enron.com'],['tim.belden@enron.com'],NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",...,False,False,False,False,False,False,False,False,False,False
1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00+00:00,['phillip.allen@enron.com'],['john.lavorato@enron.com'],Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",...,False,False,False,False,False,False,False,False,False,False
2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00+00:00,['phillip.allen@enron.com'],['leah.arsdall@enron.com'],Re: test,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,...,False,False,False,False,False,False,False,False,False,False
3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00+00:00,['phillip.allen@enron.com'],['randall.gay@enron.com'],NaN,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,...,False,False,False,False,False,False,False,False,False,False
4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00+00:00,['phillip.allen@enron.com'],['greg.piper@enron.com'],Re: Hello,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,...,False,False,False,False,False,False,False,False,False,False


In [108]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

df[df['Message-ID'] == '<6687502.1075851837002.JavaMail.evans@thyme>']

def is_from(df, from_email):
	return df['From'].apply(lambda x: any(from_email in e for e in x)) | df['X-From'].str.contains(from_email, na=False)

def is_to(df, to_email):
	return df['To'].apply(lambda x: x==x and any(to_email in e for e in x)) | df['X-To'].str.contains(to_email, na=False)

def between(df, from_email, to_email):
	return (
		(is_from(df, from_email) & is_to(df, to_email)) |
		(is_to(df, from_email) & is_from(df, to_email))
	)

df[df['Message-ID'] == '<1223589.1075857188046.JavaMail.evans@thyme>']
# df[df['Subject'].str.contains('LV Cogen Non-Recourse Debt', na=False)]['content']
# df[is_from(df, 'lnemec')]
# df[is_to(df, 'lnemec')]

# dump_to_disk(df[is_to(df, 'lnemec')], '/Users/bill/experiments/enron-analysis/outputs/lnemec_to')

# df[df['From'].apply(lambda x: 'mike.carson@enron.com' in x)]

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,result,content_no_attachment,user_prompt,token_count_total,num_tags,GUILT,LEGAl_ADVICE,LOGISTICS,ROMANCE,911,LEGAL_ADVICE,GOSSIP,MENTORSHIP,FUNNY,SPAM,PROMOTIONAL,PERSONAL,BUSINESS,FINANCIAL,>,POLITICS,LOGISTICS,HARASSMENT,FRAUD
379386,<1223589.1075857188046.JavaMail.evans@thyme>,2000-03-13 09:41:00+00:00,[stuart.zisman@enron.com],[oscar.dalton@enron.com],Re: GT Bid matrix,Stuart Zisman,Oscar Dalton,"Ozzie Pagan, Benjamin Rogers, Russ Porter, Janet R Dietrich, Michael Miller",NaN,\Benjamin_Rogers_Dec2000_2\Notes Folders\Discussion threads,Rogers-B,brogers.nsf,"Was this xls spreadsheet sent to all bidders? If so, we might have some \nexplaining to do regarding how we undercut our supplier and more importantly \nhow we could be in a position to supply (flip) a turbine. Stuart",rogers-b,"{'result': ParseResult(tags=['FRAUD', 'LOGISTICS']), 'error': None}","Was this xls spreadsheet sent to all bidders? If so, we might have some \nexplaining to do regarding how we undercut our supplier and more importantly \nhow we could be in a position to supply (flip) a turbine. Stuart","EMAIL CONTENT:\nWas this xls spreadsheet sent to all bidders? If so, we might have some \nexplaining to do regarding how we undercut our supplier and more importantly \nhow we could be in a position to supply (flip) a turbine. Stuart",309,2,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [107]:
# From: daniel.diamond@enron.com
# Sent: Monday, October 29, 2001 03:41 PM
# To: martin.cuilla@enron.com
dump_to_disk(
	df[
		(is_from(df, 'daniel.diamond') & is_to(df, 'martin.cuilla')) |
		(is_from(df, 'martin.cuilla') & is_to(df, 'daniel.diamond'))
	],
	'daniel_diamond_martin_cuilla'
)

In [113]:
# lisa.nemec@enron.com
# lnemec
# gerald.nemec@enron.com

dump_to_disk(
	df[
		between(df, 'lisa.nemec', 'gerald.nemec') |
		between(df, 'lnemec', 'gerald.nemec')
	].sort_values('Date'),
	'outputs/gerald-lisa',
	lambda x: x['Date'].timestamp()
)


In [105]:
dump_to_disk(df[is_from(df, 'michelle.cash@enron.com')], '/Users/bill/experiments/enron-analysis/outputs/michelle.cash_to')

In [116]:
row = df[
	df['content'].str.contains("Gerald - I don't know exactly what I am doing these days", case=False)
].iloc[0]

date_pretty = row['Date'].strftime('%A, %B %d, %Y %I:%M %p')

metadata = f"""Message-ID: {row['Message-ID']}
From: {'; '.join(handlena(row['From'], []))}
Sent: {date_pretty}
To: {'; '.join(handlena(row['To'], []))}
Subject: {handlena(row['Subject'], '')}"""

print(metadata)
print(row['content'])

Message-ID: <19298607.1075842584460.JavaMail.evans@thyme>
From: lnemec@ect.enron.com
Sent: Friday, November 05, 1999 09:15 PM
To: gerald.nemec@enron.com
Subject: ?
Gerald - I don't know exactly what I am doing these days and after lunch 
today,
I feel even more clueless.  I need you to know that I never wanted or thought 
in
a million years that this would be happening to us.  I also need you to know
that I have never felt so low, this is not easy for me AT ALL.  I can't bring
myself to sign the divorce papers.  This all seems so unreal to me.  I am 
sorry
I doubted your capacity to forgive me early on, but I just couldn't imagine 
that
you would ever be able to live with me again after knowing what I did.  I 
wish I
had done things differently and handled myself better.

It was good to see you and have a normal conversation like we used to.  I hope
you have a fun weekend.  See you Wednesday.

Lisa



In [122]:
media_df2 = joblib.load('/Users/bill/Downloads/enron_media_deduped.joblib')
media_df2.head()
media_df2[
	media_df2['final_path'].str.contains('37250', case=False)
]

,file_name,source_archive,filetype,output_path,md5,final_path,img_shape,smallest,embedding
